# Day05 JTAG Hacking

- 중간 보스급
- JTAG 기초 설명
- TAPC 상태도의 이해
- Atmega128 JTAG 해킹 실습


## JTAG 개요

- 하드웨어 디버깅 표준인 JTAG의 개념에 대해 이해한다
- AVR을 이용하여 JTAG을 직접 제어하는 도구를 개발해 봄으로써 작동원리를 이해한다
- 동적 디버깅, Flash Memory Dump 등 각종 JTAG 활용 예제에 대해 알아본다

## JTAG의 개념

- marionette(꼭두각시)
- CPU를 마음대로 조정할 수 있음


## 하드웨어 디버깅

- 일반적인 디버깅 방법
  - 멀티 테스터를 이용하여 일일이 확인해 본다
  - 간단한 테스트 프로그램을 만들어 확인해 본다
  - Logic Analyzer로 신호를 분석해 본다
- 문제점
  - 핀 간격이 너무 촘촘해서 멀티 테스터를 사용하기 어려운 경우
  - BGA 타입과 같이 포인트를 잡을 수 없는 경우

## JTAG의 발명

- Joint Test Action Group
  - 기술이름이자 동시에 그룹이름
  - 하드웨어 디벅ㅇ의 표준
  - 1990년: IEEE에서 표준화
- 기능
  - Pin 접촉 상태 테스트
  - Firmware 코들르 line by line으로 디버깅 가능
  - Firmware dump, writing 가능
  - 그 외 하드웨어적인 모든 것 가능

## 기본 아이디어

- MCU의 각각의 핀을 하나씩 더 빼낸다
- 해당 핀의 현재 상태를 모니터링하기 위한 목적의 작은 회로를 추가한다.
- 이것을 JTAG에서는 CELL 이라고 한다.

## IR과 DR

- Instruction Register
  - 4비트 크기
  - JTAG을 어떤 모드로 작동시킬 것인가?(이것만 신경쓰면 나머지는 자동으로 결정됨)
  - 외부로 노출된 TDI와 TDO를 내부의 어디에 연결시킬지 결정한다
  - 연결 대상: 내부 Boundary Cell 혹은 특정 레지스트
- Data Register
  - 실제 데이터 전송을 위한 레지스터
  - 위 IR에 의해 연결된 대상의 값이 DR에 저장된다.

## bypass

- 내가 보낸게 정확히 나온지
- 얘 jtag과 다른 jtag과 연동할 수 있다. 2개는 안 쓰고 1개는 쓰고 하면 됨

- clock: portA0
- pa0 -> tck
- pb0 -> td5


- PORTA = TCK
- PORTB = TDO
- PORTC = TMS
- PORTD = TDI
- GND = GND
- rabbit보드의 전원은 usb

### 용어 정리

- TCK: Test Clock, 이동
- TDO: Test Data Out
- TMS: Test Mode Select, 방향 전환
- TDI: Test Data In
- TMS와 TCK는 TAPC의 상태를 변화시킨다.


- TCK: Clock 제공(해커 -> 대상 장비)

## 팁

- SHIFT IR을 할 때 
- 값을 보낼 때는 rising edge
- 받는 것은 것은 falling edge
- 딜레이도 주는 것이 좋음. 나중에 줄여나가면 됨

## 시나리오

- 목표: JTAG ID 코드 읽기
- 제일 처음 시작이 test logic reset 이기 때문에 여기에서 tms = 0 으로 run test idle
- run test idle에서 tms = 1로 세팅해서 다음으로 진행
- select DR scan에서 tms = 0으로 세팅해서 다음으로 진행
- capture DR에서 tms = 0으로 세팅해서 다음으로 진행
- shift DR에서 id값을 읽어올만큼 반복한다.

- TMS의 모드가 어떤게 있는지 알아야? IR로 선택할지 DR로 선택할지 알 수가 있지?
- JTAG을 어떤 모드로 작동시킬 것인가???
- Instruction의 종류: 0000

## IDCODE 읽기 과정 요약

- IR에 IDCODE 명령 패턴을 집어 넣는다.
  - SHIFT-IR, Update-IR 이용
- DR 값을 읽어서 꺼내온다.

### ID 레지스터란?

- Identification Register
- 32비트
- 비트 0: IDCODE 존재 여부
- 비트 1~11: 제조사
- 비트 12~26: 부품 번호
- 비트 27~31: 버전 번호
- 32비트만 읽어와서 4개로 쪼갠 후에 뿌려주면 될 것 같음

### IDCODE 요청

- IR 레지스터
  - 0001 이 되어야 함
  - 즉, JTAG의 명령 모드를 0001로 바꾸어야 함
- 어떻게?
  - TAPC의 상태를 Shift-IR까지 옮긴 후,
  - TDI에 0001을 순서대로 입력한다.
    - 최하위 비트부터 입력, 즉 반대로 1000
- 그럼 DR 레지스터에 IDCODE가 담긴다

### IDCODE 획득

- SHIFT-DR로 이동
  - DR <-> TDI/TDO 연결
- TCK가 1 -> 0으로 변할 때 값이 나옴(falling edge에서 값을 얻는다..? 그럼 이걸 어떻게 체크하지?)
- TDI에 아무값(그냥 0)을 32번 밀어 넣어줘서 32비트의 IDCODE가 나온게 만든다.

## SHIFT-IR과 SHIFT-DR

- TMS=0일 때 자신을 다시 실행한다.
- 즉, 자신을 다시 실행할 때마다 값이 shift 된다.
- 이 때 새로운 값을 넣어주면(TDI) 동시에 기존 값이 밖으로(TDO) 출력됨

## 무엇을 어디로 Shift 시키는가?

### SHIFT-IR

- TDI가 IR로 연결된다.
- 즉, 우리가 보내는 입력이 IR로 전달된다.
- 동시에 기존값이 TDO로 나온다.

### SHIFT-DR

- TDI가 DR로 연결된다.
- 즉, 우리가 보내는 입력이 DR로 전달된다.
- 동시에 기존값이 TDO로 나온다.

- msb와 lsb를 먼저 보낼지 데이터 시트를 확인해 봐야 한다. lsb 부터 넣어야 한다.(반대로)

- data 레지스터가 실제는 가상의 레지스터. 셀제로는 존재하지 않는

In [1]:
!wget http://grayhash.com/training/jtag.c

--2015-05-01 12:38:20--  http://grayhash.com/training/jtag.c
Resolving grayhash.com... 54.65.63.9
Connecting to grayhash.com|54.65.63.9|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2637 (2.6K) [text/x-csrc]
Saving to: 'jtag.c'

100%[======================================>] 2,637       --.-K/s   in 0s      

2015-05-01 12:38:21 (16.5 MB/s) - 'jtag.c' saved [2637/2637]



```c
#define F_CPU 16000000UL
#include <avr/io.h>
#include <string.h>
#include <util/delay.h>
#include <avr/interrupt.h>

void Puts(char *data){
	int i;
	for(i=0; i<strlen(data); i++)	{
		while((UCSR0A & 0x20) == 0);
		UDR0 = data[i];
	}
}

/*
	PORTA = tck
	PORTB = TDO
	PORTC = TMS
	PORTD = TDI
	
	
	1. 래빗 보드를 껐다가 킨다. (RESET으로는 JTAG 초기화가 안 되더라)
	2. JMOD를 리셋한다.
	3. IDCODE를 확인한다.
	
	
*/


int main(void)
{
	int i, readBit;
	char str[80];
	
	DDRA = 0xFF;
	DDRB = 0x00;
	DDRC = 0xFF;
	DDRD = 0xFF;
	
	// UART 초기화
	UCSR0A = 0x00; UCSR0B = 0x98; UCSR0C = 0x06; UBRR0H = 0;UBRR0L = 8; // bps == 115200
	
	Puts("start!\r\n");
	

	PORTA = 0x00;  // ___
	
	// to RUN TEST IDLE STATE
	PORTC = 0x00;
	PORTA = 0xFF;  // ___---
	PORTA = 0x00;  // ___---___
	
	////////////////////////////////////// SHIFT-IR ///////////////////
	
	// select DR SCAN STATE
	PORTC = 0x01;
	PORTA = 0xFF;  // ___---
	PORTA = 0x00;  // ___---___
	
	// to select IR	STATE
	PORTC = 0x01;
	PORTA = 0xFF;  // ___---
	PORTA = 0x00;  // ___---___
	
	// to CAPTURE-IR STATE
	PORTC = 0x00;
	PORTA = 0xFF;  // ___---
	PORTA = 0x00;  // ___---___
	
	// to SHIFT-RO STATE
	PORTC = 0x00;
	PORTA = 0xFF;  // ___---
	PORTA = 0x00;  // ___---___


	////////////////////////////////////// INSERT INSTRUCTION ///////////////////
		
	// LSB
	PORTD = 0x01;
	PORTC = 0x00;
	PORTA = 0xFF;  // ___---
	PORTA = 0x00;  // ___---___
	
	// LSB
	PORTD = 0x00;
	PORTC = 0x00;
	PORTA = 0xFF;  // ___---
	PORTA = 0x00;  // ___---___
	
	// LSB
	PORTD = 0x00;
	PORTC = 0x00;
	PORTA = 0xFF;  // ___---
	PORTA = 0x00;  // ___---___
	
	// LSB + EXTI-IR
	PORTD = 0x00;
	PORTC = 0xFF;
	PORTA = 0xFF;  // ___---
	PORTA = 0x00;  // ___---___
	
	/////////////////////////////////////////////////////////  TO RUN AGAIN
	
	
	// to UPDATE-IR STATE
	PORTC = 0xFF;
	PORTA = 0xFF;  // ___---
	PORTA = 0x00;  // ___---___
	
	// to RUN STATE
	PORTC = 0x00;
	PORTA = 0xFF;  // ___---
	PORTA = 0x00;  // ___---___
		
	
	//////////////////////////////////////////////////////////
	
	// select DR SCAN STATE
	PORTC = 0x01;
	PORTA = 0xFF;  // ___---
	PORTA = 0x00;  // ___---___
	
	// to CAPTURE DR STATE
	PORTC = 0x00;
	PORTA = 0xFF;  // ___---
	PORTA = 0x00;  // ___---___
	
	// to SHIFT-DR STATE
	PORTC = 0x00;
	PORTA = 0xFF;  // ___---
	PORTA = 0x00;  // ___---___
	
	readBit = PINB & 1;
	sprintf(str, "readbit : %d\r\n", readBit);
	Puts(str);

	for(i=0; i<31; i++)
	{
		// to SHIFT-DR STATE
		PORTC = 0x00;
		PORTA = 0xFF;  // ___---
		PORTA = 0x00;  // ___---___
		
		readBit = PINB & 1;
		
		sprintf(str, "readbit : %d\r\n", readBit);
		Puts(str);
	}
}
```

In [6]:
s = '''11111000000010000001110100110101'''

In [9]:
ss = ''

In [10]:
for i in reversed(s):
    ss += i

In [13]:
ss

'10101100101110000001000000011111'

In [15]:
hex(ord(0b10101100101110000001000000011111))

TypeError: ord() expected string of length 1, but int found

## 주의사항

- JMOD 3.3v로 바뀌었는지 확인할 것
- ISP 모드인지 UART 모드인지 확인할 것
- 